
# 📊 Análisis de Evasión de Clientes en TelecomX LATAM

Este notebook documenta el análisis completo de los datos de clientes de **TelecomX**.  
El objetivo es comprender el fenómeno de **evasión de clientes (Churn)** y proponer estrategias de retención.

---

## 🔹 Objetivos del análisis
1. Cargar y limpiar los datos desde la API.  
2. Comprender la estructura del dataset y sus variables.  
3. Identificar problemas de calidad de datos y aplicar correcciones.  
4. Realizar un análisis exploratorio de datos (EDA) con métricas descriptivas y visualizaciones.  
5. Extraer conclusiones e insights clave para la retención de clientes.  
6. Elaborar un informe final con recomendaciones estratégicas.  


In [ ]:

# ========================================
# Importar librerías
# ========================================
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Configuración de gráficos
plt.style.use('seaborn-v0_8')
sns.set_theme()


In [ ]:

# ========================================
#  Cargar datos desde la API
# ========================================
url = "https://raw.githubusercontent.com/sthemonica/alura-voz/refs/heads/main/Dados/Telco-Customer-Churn.json"

response = requests.get(url)
data = response.json()

# Normalizar datos JSON
df = pd.json_normalize(data)

# Vista inicial
df.head()


In [ ]:

# ========================================
# Exploración inicial del dataset
# ========================================
print("Información general:")
print(df.info())

print("\nValores únicos por columna:")
for col in df.columns:
    print(f"{col}: {df[col].nunique()} únicos")


In [ ]:

# ========================================
# Limpieza y tratamiento de datos
# ========================================

# Reemplazar espacios vacíos por NaN
df = df.replace(r'^\s*$', pd.NA, regex=True)

# Eliminar duplicados
df = df.drop_duplicates()

# Conversión de columnas numéricas
df['account.Charges.Total'] = pd.to_numeric(df['account.Charges.Total'], errors='coerce')
df['account.Charges.Monthly'] = pd.to_numeric(df['account.Charges.Monthly'], errors='coerce')

# Eliminar filas sin valor en Churn
df = df[df['Churn'].notna()]

# Unificación de categorías inconsistentes
cols_internet = [c for c in df.columns if 'internet' in c]
for col in cols_internet:
    df[col] = df[col].replace({'No internet service': 'No'})

# Crear columna Cuentas_Diarias
df['Cuentas_Diarias'] = df['account.Charges.Monthly'] / 30

df.info()


In [ ]:

# ========================================
# Análisis descriptivo de datos numéricos
# ========================================
print("📊 Estadísticas descriptivas:")
display(df.describe())

print("\n📊 Mediana:")
print(df.median(numeric_only=True))

print("\n📊 Desviación estándar:")
print(df.std(numeric_only=True))


In [ ]:

# ========================================
#  Distribución de Churn
# ========================================
churn_counts = df['Churn'].value_counts(normalize=True) * 100
print("Distribución de Churn (%):")
print(churn_counts)

# Gráfico circular
fig = px.pie(df, names="Churn", title="Proporción de clientes: Retención vs Evasión")
fig.show()


In [ ]:

# ========================================
#  Churn según variables categóricas
# ========================================
categoricas = ['customer.gender', 'account.Contract', 'account.PaymentMethod', 
               'phone.PhoneService', 'internet.InternetService', 'internet.StreamingTV']

for col in categoricas:
    fig = px.histogram(df, x=col, color="Churn", barmode="group", text_auto=True,
                       title=f"Evasión por {col}")
    fig.show()


In [ ]:

# ========================================
#Churn según variables numéricas
# ========================================
numericas = ['customer.tenure', 'account.Charges.Monthly', 'account.Charges.Total', 'Cuentas_Diarias']

for col in numericas:
    fig = px.box(df, x="Churn", y=col, color="Churn", 
                 title=f"{col} según Churn")
    fig.show()

# Dispersión tenure vs cargos mensuales
fig = px.scatter(df, x="customer.tenure", y="account.Charges.Monthly", 
                 color="Churn", title="Tenure vs Cargos mensuales por Churn")
fig.show()
